# Tools

In [ ]:
import pandas as pd

In [ ]:
from IPython.display import display

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
import numpy as np

In [ ]:
from matplotlib import cm

In [ ]:
from itertools import combinations

In [ ]:
from matplotlib import colormaps

In [ ]:
import networkx as nx

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
import plotly.graph_objects as go

In [ ]:
import seaborn as sns

# Analysis Sample description

### 1 Import, view and exclude data

#### 1.1 Import data

In [ ]:
df = pd.read_excel("MRVdata_0810.xlsx")
pd.options.display.max_columns = None
#display(df)
#code above: shows excel table

#### 1.2 Overview of data

In [ ]:
df['Inclusion/exclusion'].value_counts() 
#--> 205 studies included, 180 excluded, 14 no access

#### 1.3 Exclude data

In [ ]:
# filter data frame to exclude studies from 2024
df_filtered = df[df['Year'] != 2024]

In [ ]:
df_included = df_filtered[df_filtered['Inclusion/exclusion'] == 'Inclusion'] # new dataframe defined to exclude studies without access or excluded papers
df_included['Inclusion/exclusion'].value_counts()
#code above: controll new data frame

### 2 Study location

#### 2.1 Overview of included study locations

In [ ]:
df_included['Study location'].value_counts()
#Results: most common study locations are not specified, multiple countries, US, China and Australia

In [ ]:
df_included['Study location: specification'].value_counts()
# Result: No specific pattern visible at first glance. Often neighbouring countries or definitions of maritime regions

#### 2.2 Grouping of study location

In [ ]:
# read excel file with continent data 
continent_data = pd.read_excel('continents.xlsx')

# create a dictionary for the assignment of countries to continents
country_to_continent = dict(zip(continent_data['Country'], continent_data['Continent']))

# Function to determine the continent based on the country
def get_continent(study_location, continent_column): #part of the function, not column names of the df; will be defined later when function is applied
    if pd.isna(study_location):
        return "Not specified"
    if study_location == "Multiple countries":
        return continent_column  # uses the manually definied continents for the category "Multiple countries"
    return country_to_continent.get(study_location.strip(), "Not specified")  # Search for the country in the dictionary and return the continent or 'not specified'

# avoid warning
df_included = df_included.copy()

# apply function to df_included
df_included['Study location_continent'] = df_included.apply(
    lambda row: get_continent(row['Study location'], row['Study location: Continent']),
    axis=1
)

#pd.options.display.max_columns = None
#display(df_included)

In [ ]:
df_included['Study location_continent'].value_counts()

In [ ]:
desired_order_continents = ['Africa', 'Asia', 'Europe', 'North America', 'South America', 'Oceania', 'USA', 'China', 'Global']

df_included['Study location_continent'] = pd.Categorical(df_included['Study location_continent'], categories=desired_order_continents, ordered=True)

#### Figure 1: Number of MRV publications per study location

In [ ]:
#papers per study location
papers_per_location= df_included['Study location_continent'].value_counts().sort_index()

# bar chart for papers per year
plt.figure(figsize=(10, 5))
papers_per_location.plot(kind='bar', color='skyblue')
plt.title('Number of MRV publications per Study Location')
plt.xlabel('Continent/Country')
plt.ylabel('Publications')
plt.grid(axis='y')
plt.show()

### 3 Publication date

#### 3.1 Overview of included publication dates

In [ ]:
df_included.groupby('Year').size() # Results: most of our included studies were published 2019 and later

#### Figure 2: Number of MRV Publications per year

In [ ]:
# Convert years to integer to avoid .0 and ensure that years are shown correctly in the figure
df_included['Year'] = df_included['Year'].astype(float).astype(int)

In [ ]:
#papers per year
papers_per_year = df_included['Year'].value_counts().sort_index()

# bar chart for papers per year
plt.figure(figsize=(10, 5))
papers_per_year.plot(kind='bar', color='skyblue')
plt.title('Number of MRV publications per year')
plt.xlabel('year')
plt.ylabel('publications')
plt.grid(axis='y')
plt.show()

### 4 CDR method and study focus

#### 4.1 Definition CDR methods & color scheme

In [ ]:
#columns with all CDR methods
CDR_columns = ['BECCS', 'General forestry', 'A/R', 'Agroforestry', 'Forest management', 'DACCS', 'EW', 'OAE', 'OIF/AU', 'Biochar', 'SCS', 'Blue carbon', 'Restoration of landscapes and peats', 'CCUS', 'Other', 'General CDR', 'DOC', 'Algae ', 'CCS']

In [ ]:
# Define the custom color scheme for each CDR method
custom_colors_CDR_method = {
    'BECCS': '#f4d6ae', 'General forestry': '#bcbd22', 'A/R': '#aab792', 
    'Agroforestry': '#e9ede2', 'Forest management': '#d2bfac', 'DACCS': '#dd7500',
    'EW': '#d6b8c1', 'OAE': '#ecf1f9', 'OIF/AU': '#709fca', 'Biochar': '#b7768bff',
    'SCS': '#769112', 'Blue carbon': '#d8e3f1', 'Restoration of landscapes and peats': '#d3dbc7',
    'CCUS': '#b55d00', 'Other': '#e5e5e5', 'General CDR': '#e5b138', 'DOC': '#17becf',
    'Algae ': '#2ca02c', 'CCS': '#d62728'
}

#### 4.2 Overview of CDR methods & foci

In [ ]:
# nummber of main focus per CDR method
CDR_maintopic = (df_included[CDR_columns] == 1).sum()
CDR_maintopic_sorted = CDR_maintopic.sort_values(ascending=False)
#print(CDR_maintopic_sorted)
# Result: A/R, SCS and EW are the most common CDR side topics used in the context of MRV

In [ ]:
#number of side topic per CDR method
CDR_sidetopic = (df_included[CDR_columns] == 2).sum()
CDR_sidetopic_sorted = CDR_sidetopic.sort_values(ascending=False)
#print(CDR_sidetopic_sorted)
# Result: A/R, SCS and EW are the most common CDR side topics used in the context of MRV

In [ ]:
# number of framings per CDR methods
CDR_framing = (df_included[CDR_columns] == 3).sum()
CDR_framing_sorted = CDR_framing.sort_values(ascending=False)
#print(CDR_framing_sorted)
# Result: BECCS, FM, Agroforestry, SCS and blue carbon rarely used as framings in the context of MRV

In [ ]:
# Results for all CDR technologies regardless if it is mentioned as main/sidetopic or framing
CDR_method = (df_included[CDR_columns] > 0).sum()
CDR_method_sorted = CDR_method.sort_values(ascending=False)
#print(CDR_method_sorted)
# Decided to analyse year of publication, study location, etc. per CDR method, regardless of whether the method is only mentioned or the main topic of the articles (even if it is mentioned, the author had the intention to include the results of the study for that method)

#### Figure 3: Shares of CDR Methods in the Sample

In [ ]:
#Overview graphic for all CDR methods in the color scheme

# Generate a list of colors based on the sorted CDR methods
colors_CDR_method = [custom_colors_CDR_method[method] for method in CDR_method_sorted.index]

# Set the size of the plot
plt.figure(figsize=(10, 6))

# Create the horizontal bar plot with custom colors and assign hue to the y variable
sns.barplot(x=CDR_method_sorted.values, y=CDR_method_sorted.index, hue=CDR_method_sorted.index, palette=colors_CDR_method, dodge=False, legend=False)

# Add the percentages behind the bars
total = CDR_method_sorted.sum()
for i, value in enumerate(CDR_method_sorted):
    percentage = 100 * value / total
    plt.text(value + 1, i, f'{percentage:.1f}%', va='center')  # Add 1 to move the percentage text slightly away from the bar

# Set plot labels and title
plt.xlabel('Count of CDR Methods')
plt.ylabel('CDR Method')
plt.title('Count of CDR Methods by Type')

# Show the plot
plt.tight_layout()
plt.show()

In [ ]:
#add new column for CDR method and study focus in the dataset
def extract_CDR_method_focus(row):
    CDR_method = []
    CDR_focus = []
    
    for column in CDR_columns:  # previously defined list with columns that contain CDR methods
        if row[column] > 0:
            CDR_method.append(column)
            CDR_focus.append(row[column])
    
    return pd.Series([CDR_method, CDR_focus])

# new copy of the data set to avoid warning
df_included = df_included.copy()

# add new columns to the data frame
df_included['CDR_method'] = df_included.apply(lambda row: extract_CDR_method_focus(row)[0], axis=1)
df_included['CDR_focus'] = df_included.apply(lambda row: extract_CDR_method_focus(row)[1], axis=1)

#display dataset to see if newly added columns are correct
#pd.options.display.max_columns = None
#display(df_included)

### 5 MRV topics and study focus

#### 5.1 Definition MRV groups & color scheme

In [ ]:
#columns with all MRV topics
MRV_columns = ['General MRV', 'Quantification', 'Monitoring', 'Quality', 'Governance', 'Reporting', 'Verification', 'External impacts']

In [ ]:
# Define the new custom color scheme for MRV topics
mrv_topic_colors = {
    'General MRV': '#e5b138',      # muted yellow
    'Quantification': '#7da7b8',   # darker bluish-grey
    'Monitoring': '#8dc39f',        # stronger soft green
    'Quality': '#c76a85',           # berry red
    'Governance': '#b39bc8',        # stronger lavender
    'Reporting': '#e1a6b8',         # stronger soft pink
    'Verification': '#a8a8a8',      # medium grey
    'External impacts': '#d0c97b'   # stronger soft yellow
}

#### 5.2 Overview of MRV topics & foci

In [ ]:
# nummber of main focus per MRV study topics
MRV_maintopic = (df_included[MRV_columns] == 1).sum()
MRV_maintopic_sorted = MRV_maintopic.sort_values(ascending=False)
#print(MRV_maintopic_sorted)
# Result: Quantification is the main topic (in 165 studies), followed by Monitoring (18 studies) and Quality (15 studies)

In [ ]:
# nummber of side focus per MRV study topics
MRV_maintopic = (df_included[MRV_columns] == 2).sum()
MRV_maintopic_sorted = MRV_maintopic.sort_values(ascending=False)
#print(MRV_maintopic_sorted)
# Result: Quantification is also the the main side focus (in 18 studies), followed by Monitoring (18 studies) and Quality (17 studies)

In [ ]:
# nummber of all focuses per MRV study topics
MRV_topic = (df_included[MRV_columns] > 0).sum()
MRV_topic_sorted = MRV_topic.sort_values(ascending=False)
#print(MRV_topic_sorted)

#### Figure 4: Distribution of MRV topics

In [ ]:
#Overview graphic for all MRV topics 

# Calculate the counts for each MRV topic
mrv_topic_counts = MRV_topic_sorted

# Create the pie chart
fig, ax = plt.subplots(dpi=150, figsize=(8, 6))

# Create the pie chart without percentages displayed
wedges, texts = ax.pie(mrv_topic_counts, 
                       labels=None, 
                       colors=[mrv_topic_colors[topic] for topic in mrv_topic_counts.index],
                       startangle=90,
                       textprops=dict(color="black"))

# Add a white circle in the middle to make it a donut chart
white_circle = plt.Circle((0, 0), 0.70, color='white')
ax.add_artist(white_circle)

# Create legend labels with percentages
legend_labels = [f'{topic} ({100 * count / mrv_topic_counts.sum():.1f}%)' for topic, count in zip(mrv_topic_counts.index, mrv_topic_counts)]

# Place the legend next to the pie chart
ax.legend(wedges, legend_labels, 
          title="MRV Topics", loc="center left", bbox_to_anchor=(1, 0, 0.5, 1))

# Set plot title
plt.title('Distribution of MRV Topics')
plt.axis('equal')  # Equal aspect ratio ensures that pie is drawn as a circle.

# Show the plot
plt.tight_layout()

In [ ]:
#add new column for MRV topic and study focus to the dataset
def extract_MRV_topic_focus(row):
    MRV_topic = []
    MRV_focus = []
    
    for column in MRV_columns:  # previously defined list with columns that contain MRV topics
        if row[column] > 0:
            MRV_topic.append(column)
            MRV_focus.append(row[column])
    
    return pd.Series([MRV_topic, MRV_focus])

# new copy of the data set to avoid warning
df_included = df_included.copy()

# add new columns to the data frame
df_included['MRV_topic'] = df_included.apply(lambda row: extract_MRV_topic_focus(row)[0], axis=1)
df_included['MRV_focus'] = df_included.apply(lambda row: extract_MRV_topic_focus(row)[1], axis=1)

#display dataset to see if newly added columns are correct
#pd.options.display.max_columns = None
#display(df_included)

### 6 Exploded dataset: CDR group

#### 6.1 Datset explosion

In [ ]:
# explode CDR method and focus
df_CDRexplode = df_included.explode(['CDR_method', 'CDR_focus'])

# explode MRV topic and focus
df_MRVexplode = df_included.explode(['MRV_topic', 'MRV_focus'])

# explode MRV topic and focus
df_exploded = df_CDRexplode.explode(['MRV_topic', 'MRV_focus'])

# every CDR method and every MRV topic is in one line (in combination with study focus)

#display dataset to see if newly added columns are correct
#pd.options.display.max_columns = None
#display(df_exploded)

#### 6.2 Definition CDR groups & color scheme

In [ ]:
# Group CDR Methods to marine, terrestrial, and CCS technologies. Keep A/R and SCS as individual technologies
cdr_group_mapping = {
     # Marine CDR group
    "OIF/AU":"Marine CDR",
    "Blue carbon": "Marine CDR",
    "OAE": "Marine CDR",
    "DOC": "Marine CDR",
    "Algae ": "Marine CDR",
    
    # Terrestrial CDR group
    "EW": "Terrestrial CDR",
    "Biochar": "Terrestrial CDR",
    "Restoration of landscapes and peats": "Terrestrial CDR",
    
    # CCS group
    "DACCS": "CCS",
    "BECCS": "CCS",
    "CCUS": "CCS",
    "CCS": "CCS",
    
    # General CDR/Other group
    "General CDR": "General CDR/Other",
    "Other": "General CDR/Other",

    # SCS
    "SCS": "SCS",

    # Forestry
    "General forestry": "Forestry",
    "A/R": "Forestry",
    "Agroforestry": "Forestry",
    "Forest management": "Forestry",
}
# Create a new column ‘CDR_group’ in df_exploded based on the ‘CDR_method’ column
df_exploded['CDR_group'] = df_exploded['CDR_method'].map(cdr_group_mapping)

# Display the updated DataFrame
#pd.options.display.max_columns = None  # Show all columns
#display(df_exploded)

In [ ]:
# Define the custom color scheme for each CDR group
cdr_group_colors = {
    "Marine CDR": '#7eaed3',    # bluew for Marine CDR
    "Terrestrial CDR": '#b2df8a',  # green for Terrestrial CDR
    "CCS": '#ff7f00',            # orange for CCS
    "General CDR/Other": '#e5e5e5', # grey for General/Other
    "SCS": '#C1A28A' ,            # brown for SCS
    "Forestry": '#8fbc8f'         # green for Forestry
}


#### Figure 5: Distribution of CDR groups

In [ ]:
# Calculate the counts for each CDR group
cdr_group_counts = df_exploded['CDR_group'].value_counts()

# Define the labels with percentages in parentheses
labels = [f'{group} ({percentage:.1f}%)' for group, percentage in zip(cdr_group_counts.index, 
                                                                      100 * cdr_group_counts / cdr_group_counts.sum())]

# Set the colors for each group
colors = [cdr_group_colors[group] for group in cdr_group_counts.index]

# Create the pie chart
fig, ax = plt.subplots(dpi=150, figsize=(6, 6))

plt.pie(cdr_group_counts, labels=labels, colors=colors, startangle=90, labeldistance=1.05)

# Add a white circle in the middle to make it a donut chart
white_circle = plt.Circle((0, 0), 0.70, color='white')
p = plt.gcf()
p.gca().add_artist(white_circle)

# Set plot title
plt.title('Distribution of CDR Groups')

# Show the plot
plt.tight_layout()
plt.show()

### 7 Research Methods (focus on qualitative methods)

#### 7.1 Overview of primary research methods

In [ ]:
# Define the custom color scheme for research methods
research_methods_colors = {
    'Modelling': '#81C3D7',  # soft turquoise
    'Experiment - Field study': '#a68e71',  # tan '
    'Remote sensing': '#5BC0EB',  # soft blue
    'Qualitative research': '#FF9F40',  # light orange 
    'Data analysis/Statistical analysis/Econometric': '#4A90E2',   # strong blue'
    'Experiment - Laboratory experiment': '#FF6F61',  # coral red
    'Review': '#f4a460',  # sandy orange
    'Unknown': '#C0C0C0'  # grey
}

In [ ]:
# Calculate the counts for each MRV topic
research_methods_counts = df_included['Primary research method'].value_counts()

# Create the pie chart
fig, ax = plt.subplots(dpi=150, figsize=(8, 6))

# Create the pie chart without percentages displayed
wedges, texts = ax.pie(research_methods_counts, 
                       labels=None, 
                       colors=[research_methods_colors[method] for method in research_methods_counts.index],
                       startangle=90,
                       textprops=dict(color="black"))

# Add a white circle in the middle to make it a donut chart
white_circle = plt.Circle((0, 0), 0.70, color='white')
ax.add_artist(white_circle)

# Create legend labels with percentages
legend_labels = [f'{method} ({100 * count / research_methods_counts.sum():.1f}%)' for method, count in zip(research_methods_counts.index, research_methods_counts)]

# Place the legend next to the pie chart
ax.legend(wedges, legend_labels, 
          title="Research Methods", loc="center left", bbox_to_anchor=(1, 0, 0.5, 1))

# Set plot title
plt.title('Distribution of Research Methods')
plt.axis('equal')  # Equal aspect ratio ensures that pie is drawn as a circle.

# Show the plot
plt.tight_layout()

#### 7.2 Publications with a qualitative (primary) research method per CDR or MRV method

In [ ]:
df_qualiMRV = df_MRVexplode [df_MRVexplode['Primary research method'] == 'Qualitative research']

In [ ]:
df_qualiCDR = df_CDRexplode [df_CDRexplode['Primary research method'] == 'Qualitative research']

#### Figure 6 Pie Chart: Share of MRV topics for qualitative publications

In [ ]:
# Count occurrences of each CDR Technology
quali_counts_MRV = df_qualiMRV['MRV_topic'].value_counts()

# Map the colors to the CDR Technologies
quali_colors_MRV = [mrv_topic_colors[method] for method in quali_counts_MRV.index]

# Create pie chart with both percentage and absolute numbers
plt.figure(figsize=(8,8))
quali_counts_MRV.plot(
    kind='pie', 
    autopct=lambda p: f'{p:.1f}%\n({int(p * sum(quali_counts_MRV) / 100)})',  # Show percentage and absolute numbers
    startangle=140, 
    colors=quali_colors_MRV
)

plt.title('Share of MRV Topics in Publications with a Qualitative Primary Research Method')
plt.ylabel('')  
plt.show()

#### Figure 7 Pie Chart: Share of CDR topics for qualitative publications

In [ ]:
# Count occurrences of each CDR Technology
quali_counts_CDR = df_qualiCDR['CDR_method'].value_counts()

# Map the colors to the CDR Technologies
quali_colors_CDR = [custom_colors_CDR_method[method] for method in quali_counts_CDR.index]

# Create pie chart with both percentage and absolute numbers
plt.figure(figsize=(8,8))
quali_counts_CDR.plot(
    kind='pie', 
    autopct=lambda p: f'{p:.1f}%\n({int(p * sum(quali_counts_MRV) / 100)})',  # Show percentage and absolute numbers
    startangle=140, 
    colors=quali_colors_CDR 
)

plt.title('Share of CDR Technologies in Publications with a Qualitative Primary Research Method')
plt.ylabel('')  
plt.show()

#### 7.2 Publications with a qualitative (primary) research method per CDR and MRV topic

In [ ]:
# Creating a new data frame only includeing those papers that include a specific method (qualitative research)
df_quali = df_exploded.loc[df_exploded["Primary research method"].notna()]

In [ ]:
# Create a new DataFrame for rows where 'Primary research method' is 'Qualitative research'
df_quali = df_exploded[df_exploded['Primary research method'] == 'Qualitative research']

#### Figure 8 Stacked Bar Plot: Distribution of CDR methods by MRV topic for qualitative studies

In [ ]:
# Step 1: Group by 'CDR_method' and 'MRV_topic', and count occurrences
grouped_quali = df_quali.groupby(['CDR_method', 'MRV_topic']).size().unstack(fill_value=0)
set(grouped_quali)

# Step 2: Check which MRV topics are available in the grouped data
available_mrv_topics = grouped_quali.columns.intersection(mrv_topic_colors.keys()).tolist()

# Step 3: Filter the color mapping to only include the available MRV topics
method_colors = [mrv_topic_colors[topic] for topic in available_mrv_topics]

# Step 4: Plot the data using a stacked bar chart
ax = grouped_quali[available_mrv_topics].plot(
    kind='bar', 
    stacked=True, 
    color=method_colors, 
    figsize=(12, 7))
plt.xlabel('CDR Method')
plt.ylabel('Number of Publications')
plt.title('Distribution of CDR Technologies by MRV Topics')
plt.xticks(rotation=45, ha='right')
plt.legend(title='MRV Topic', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.tight_layout()
plt.show()


### 8 Deeper analysis of Studies without a specified location

#### 8.1 CDR Technologies

In [ ]:
# Creating a new data frame only includeing those papers that do now include a specific location (NaN)
df_no_location = df_CDRexplode.loc[df_CDRexplode["Study location: Continent"].notna()]

In [ ]:
# Create a new DataFrame for rows where 'Study location' is 'Not specified'
df_not_specified = df_CDRexplode[df_CDRexplode['Study location'] == 'Not specified']

#df_not_specified


#### Figure 9 Pie chart of CDR Technologies

In [ ]:
# Count occurrences of each CDR Technology
nolo_counts = df_not_specified['CDR_method'].value_counts()

# Map the colors to the CDR Technologies
nolo_colors = [custom_colors_CDR_method[method] for method in nolo_counts.index]

# Create pie chart with both percentage and absolute numbers
plt.figure(figsize=(8,8))
nolo_counts.plot(
    kind='pie', 
    autopct=lambda p: f'{p:.1f}%\n({int(p * sum(nolo_counts) / 100)})',  # Show percentage and absolute numbers
    startangle=140, 
    colors=nolo_colors
)

plt.title('Share of CDR Technologies in Publications without a specified location')
plt.ylabel('')  
plt.show()


#### Figure 10 Pie chart of Research methods

In [ ]:
# Count occurrences of each CDR Technology
nolo_counts = df_not_specified['Primary research method'].value_counts()

# Map the colors to the CDR Technologies
nolo_colors = [research_methods_colors.get(method, '#d3d3d3') for method in nolo_counts.index]  # Default to grey if method not found

# Create pie chart with both percentage and absolute numbers
plt.figure(figsize=(8,8))
nolo_counts.plot(
    kind='pie', 
    autopct=lambda p: f'{p:.1f}%\n({int(p * sum(nolo_counts) / 100)})',  # Show percentage and absolute numbers
    startangle=140, 
    colors=nolo_colors
)

plt.title('Share of Research Methods in Publications without a specified location')
plt.ylabel('')  
plt.show()

#### Figure 11 Stacked bar plot showing CDR Technology and Primary Research method

In [ ]:
# Group the data by 'Primary research method' and 'CDR_method' and count occurrences
grouped_nolo = df_not_specified.groupby(['Primary research method', 'CDR_method']).size().unstack()

# Check which CDR methods are present in the grouped_df
available_methods = grouped_nolo.columns.intersection(custom_colors_CDR_method.keys())
print("Available CDR Methods:", available_methods)

# Filter the color mapping to only include available methods
method_colors = [custom_colors_CDR_method[method] for method in available_methods if method in custom_colors_CDR_method]

# Create a stacked bar plot with only the available methods
grouped_nolo[available_methods].plot(kind='bar', stacked=True, color=method_colors, figsize=(12, 7))

# Set plot labels and title
plt.xlabel('Primary Research Method')
plt.ylabel('Number of Publications')
plt.title('Distribution of CDR Technologies by Primary Research Method')
plt.xticks(rotation=45, ha='right') 
plt.legend(title='CDR Method', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.tight_layout()
plt.show()

In [ ]:
# Group the data by 'Primary research method' and 'CDR_method' and count occurrences
grouped_nolo = df_not_specified.groupby(['CDR_method', 'Primary research method']).size().unstack()

# Create a stacked bar plot with only the available methods
grouped_nolo.plot(kind='bar', stacked=True, figsize=(12, 7), color=research_methods_colors)

# Set plot labels and title
plt.ylabel('Primary Research Method')
plt.xlabel('Number of Publications')
plt.title('Distribution of CDR Technologies by Primary Research Method')
plt.xticks(rotation=45, ha='right')  
plt.legend(title='Primary Reseach Method', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.tight_layout()
plt.show()

#### 7. 2 MRV Topics

In [ ]:
# Creating a new data frame only includeing those papers that do now include a specific location (NaN)
df_nolo_MRV = df_MRVexplode.loc[df_MRVexplode["Study location: Continent"].notna()]

In [ ]:
# Create a new DataFrame for rows where 'Study location' is 'Not specified'
df_nosp_MRV = df_MRVexplode[df_MRVexplode['Study location'] == 'Not specified']

#df_MRVexplode


#### Figure 12 Pie chart of CDR Technologies

In [ ]:
# Count occurrences of each CDR Technology
nolo_counts = df_nosp_MRV['MRV_topic'].value_counts()

# Map the colors to the CDR Technologies
nolo_colors = [mrv_topic_colors[method] for method in nolo_counts.index]

# Create pie chart with both percentage and absolute numbers
plt.figure(figsize=(8,8))
nolo_counts.plot(
    kind='pie', 
    autopct=lambda p: f'{p:.1f}%\n({int(p * sum(nolo_counts) / 100)})',  # Show percentage and absolute numbers
    startangle=140, 
    colors=nolo_colors
)

plt.title('Share of MRV Topics in Publications without a specified location')
plt.ylabel('')  
plt.show()


#### Figure 13 Stacked bar plot showing Research Methods sorted stacked by MRV Topic

In [ ]:
# Group the data by 'Primary research method' and 'CDR_method' and count occurrences
grouped_nolo = df_nosp_MRV.groupby(['Primary research method', 'MRV_topic']).size().unstack()

# Check which CDR methods are present in the grouped_df
available_methods = grouped_nolo.columns.intersection(mrv_topic_colors.keys())
print("Available MRV Topics:", available_methods)

# Filter the color mapping to only include available methods
method_colors = [mrv_topic_colors[method] for method in available_methods if method in mrv_topic_colors]

# Create a stacked bar plot with only the available methods
grouped_nolo[available_methods].plot(kind='bar', stacked=True, color=method_colors, figsize=(12, 7))

# Set plot labels and title
plt.xlabel('Primary Research Method')
plt.ylabel('Number of Publications')
plt.title('Distribution of MRV Topics by Primary Research Method')
plt.xticks(rotation=45, ha='right') 
plt.legend(title='CDR Method', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.tight_layout()
plt.show()

#### Figure 14 Stacked bar plot showing MRV Topic stacked by Primary Research method

In [ ]:
# Group the data by 'Primary research method' and 'CDR_method' and count occurrences
grouped_nolo = df_nosp_MRV.groupby(['MRV_topic', 'Primary research method']).size().unstack()

# Create a stacked bar plot with only the available methods
grouped_nolo.plot(kind='bar', stacked=True, color=research_methods_colors, figsize=(12, 7))

# Set plot labels and title
plt.ylabel('Primary Research Method')
plt.xlabel('Number of Publications')
plt.title('Distribution of MRV Topics by Primary Research Method')
plt.xticks(rotation=45, ha='right')  
plt.legend(title='Primary Reseach Method', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.tight_layout()
plt.show()

#### Figure 14 Stacked bar plot showing MRV topic and CDR Method

In [ ]:
# Creating a new data frame only includeing those papers that do now include a specific location (NaN)
df_nolo_comp = df_exploded.loc[df_exploded["Study location: Continent"].notna()]

In [ ]:
# Create a new DataFrame for rows where 'Study location' is 'Not specified'
df_nosp_comp = df_exploded[df_exploded['Study location'] == 'Not specified']

In [ ]:
# Group the data by 'Primary research method' and 'CDR_method' and count occurrences
grouped_nolo = df_nosp_comp.groupby(['CDR_method', 'MRV_topic']).size().unstack()

# Check which CDR methods are present in the grouped_df
available_methods = grouped_nolo.columns.intersection(mrv_topic_colors.keys())
print("Available MRV Topics:", available_methods)

# Filter the color mapping to only include available methods
method_colors = [mrv_topic_colors[method] for method in available_methods if method in mrv_topic_colors]

# Create a stacked bar plot with only the available methods
grouped_nolo[available_methods].plot(kind='bar', stacked=True, color=method_colors, figsize=(12, 7))

# Set plot labels and title
plt.xlabel('Primary Research Method')
plt.ylabel('Number of Publications')
plt.title('Distribution of CDR Technologies by MRV Topics')
plt.xticks(rotation=45, ha='right') 
plt.legend(title='CDR Method', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.tight_layout()
plt.show()